In [ ]:
import seaborn as sns
DATA_PATH ='/Users/favea/Downloads/swiss-data'
import pandas as pd
import wandb
import json
import numpy as np
import matplotlib.pyplot as plt

sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [8, 5]})

In [ ]:

df_delay = pd.read_csv(os.path.join(DATA_PATH,'delay.csv'), delimiter=';', dtype={'wh_fdel_delay_subcode':str})
df_fis = pd.read_csv(os.path.join(DATA_PATH,'fis.csv'), delimiter=';')
merged_df = df_fis.merge(df_delay, left_on='wh_fleg_leg_i', right_on='wh_fdel_leg_i')

merged_df['is_delayed'] = merged_df.wh_fdel_delay_time>=15 # We define delay as 15 miutes threhold of delay time 

In [ ]:
old_merged = merged_df.copy()

## Let's check the reference label

In [ ]:
# We can already see that the labels are not very balanced, it might be a problem during training.
sns.displot(
    merged_df['wh_fdel_delay_time'], kde=False, bins=100,
).set(xlabel='Delay', ylabel='Count');

In [ ]:
merged_df.groupby('is_delayed').count().iloc[:,0] # 5:1 the proportion of Not-delayed and delayed  (every 5 flight, there's one that's delayed)

### Airport factors

1. We want to add information about number of flights departing on that airport that day

In [ ]:
merged_df.wh_fleg_dep_dt_scd = pd.to_datetime(merged_df.wh_fleg_dep_dt_scd )

In [ ]:
flights_per_day = merged_df.groupby(by=['wh_fleg_dep_day_scd','wh_fleg_dep_ap_scd']).count().reset_index().iloc[:,:3].rename(columns ={'wh_fleg_leg_i':'flights_per_day'} )

In [ ]:
merged_df = pd.merge(left = merged_df, right = flights_per_day, on= ['wh_fleg_dep_day_scd','wh_fleg_dep_ap_scd'])

In [ ]:
merged_df[['flights_per_day','wh_fdel_delay_time']].corr() # does not seem correlated

2. We want information about flights departing at the same time of the flight

In [ ]:
def count_flights_within_one_hour(row):
    # Filter the DataFrame to only include rows with a departure time within one hour of the input datetime
    start_time = row["wh_fleg_dep_dt_scd"] - pd.Timedelta(hours=1)
    end_time = row["wh_fleg_dep_dt_scd"] + pd.Timedelta(hours=1)
    airport = row["wh_fleg_dep_ap_scd"]
    filtered_data = merged_df[(merged_df["wh_fleg_dep_dt_scd"] >= start_time) & (merged_df["wh_fleg_dep_dt_scd"] <= end_time) & (merged_df['wh_fleg_dep_ap_scd']==airport)]
    
    # Count the number of rows in the filtered DataFrame
    return filtered_data.count()['wh_fleg_leg_i']
    
from multiprocessing import cpu_count, pool
num_processes = cpu_count()
pool = pool.Pool(num_processes)
results = [pool.apply_async(count_flights_within_one_hour, (row,)) for index, row in merged_df.iterrows()]

# Wait for the parallel processing to finish
pool.close()
pool.join()

# Store the results of the parallel processing in a list
num_flights = [result.get() for result in results]

In [ ]:
merged_df.to_csv('/Users/favea/Downloads/swiss-data/10122022_merged.csv')

In [ ]:
flights_within_hour = pd.read_csv('/Users/favea/Downloads/swiss-data/num_flights_within_hour.csv', index_col=0)

In [ ]:
merged_df['flights_btw_1_hour'] = flights_within_hour

3. Info about previous flight on the wheel 

If a previous flight on the same wheel is delayed, it's more likely i'll have a delay.

In [ ]:
previous_delays = merged_df[['wh_fleg_leg_i','is_delayed']].copy().rename(columns={'wh_fleg_leg_i':'previous_flight','is_delayed':'previous_is_delayed'})

In [ ]:
merged_df = pd.merge(left = merged_df, right = previous_delays, how='left', left_on = 'wh_fleg_rot_leg_i_prev', right_on = 'previous_flight')

In [ ]:
merged_df.groupby(by=['previous_is_delayed','is_delayed']).count() # it makes sense indeed that if a previous flight has been delayed, we'll have a delay on the current one

In [ ]:
# there will be NaN values, we need to address that.
# Let's chedck how many have delayed label (if not too much, we can remove them altogheter)
merged_df.previous_is_delayed.isna().sum()
merged_df[merged_df.previous_is_delayed.isna()].groupby(by='is_delayed').count()

About 20% have delayed = True.
We can remove it, or we can add False if a NaN is present in previous_is_delayed (Meaning we assume no delay on preivous rotation)

Let's remove it: it can help with imbalance


In [ ]:
merged_df = merged_df[~merged_df.previous_is_delayed.isna()]

In [ ]:
merged_df = merged_df.drop_duplicates()

## Filtering the dataset

### Positioning/Turnaround flights


There are 125 flights that have departure and arrival in the same location.
The reason might be that these flights are CARRY or POSITIONING flights, not used to carry people or supplies.
Intuitively, delays should be detected also for this kind of flights, however the impact on the business and customer satisfaction is not that relevant.

In [ ]:
pd.set_option('display.max_rows', None)
positioning_flights = merged_df[merged_df.wh_fleg_dep_ap_scd == merged_df.wh_fleg_arr_ap_scd]
positioning_flights.head(10).T

In [ ]:
# Let's check how many of these positioning flights are actually late
positioning_flights.groupby(by='is_delayed').count().iloc[:,0]

Given the presence of many delayed flights, it might be important to consider them and use to train our model.

#### Cancelled Flights
If there's cancelled fligths, I can remove them since they can't be considered delayed.

In [ ]:
merged_df.groupby('wh_fleg_leg_state').count().iloc[:,0] # There's 4254 cancelled flights out of 262094

In [ ]:
merged_df.groupby(['wh_fleg_leg_state','is_delayed']).count().iloc[:,0] # There's only 16 flights among the cancelled flights that are labeled as 'delayed'. Therefore we can remove it.

cancelled_flights_idx  = merged_df.wh_fleg_leg_state == 'CNL'

In [ ]:
merged_df = merged_df[~cancelled_flights_idx]

## Features that might be useful.

I want to consider only features that do not introduce an excessive bias during inference. Namely, if I insert scheduled departure time and airborne departure time, 
it's likely that my model will be able to predict pretty well whether there's gonna be a delay. 
Hence, I want to consider only features about:

* Flights ( ORIGIN, DESTINATION, LENGTH OF THE TRIP, HOUR_OF_TRIP, DAY_OF_TRIP, SEASON_OF_TRIP, #LEGS (I.E. LAYOVERS + 1))
* MAIL/CARGO (information about loading/off-loading of cargo items or mail that can delay the departure) 
* LUGGAGES (the more the luggages, the more time is needed to load them in the airplane. Also, their weight might correlated with delay)
* PASSENGERS 



In [ ]:
merged_df.head()

In [ ]:
map_columns = { # FLIGHT INFO
                'wh_fleg_leg_i':'id_flight',
                'wh_fleg_flt_carrier': 'carrier',
                'wh_fleg_dep_ap_scd': 'origin',
                'wh_fleg_arr_ap_scd': 'destination',
                'wh_fleg_leg_dist_scd': 'distance_trip',
                'wh_fleg_season':'season_trip',
                'wh_fleg_dep_dt_scd':'scheduled_time_departure', # to parse (DAY | MONTH | HOUR)
                

                
                # MAIL/CARGO INFO
                'wh_fleg_mail':'mails_data',
                'wh_fleg_cargo':'cargo_data',
                
                # Luggages
                'wh_fleg_baggage_pieces':'number_checked_luggages',

                # Passengers
                'wh_fleg_pax_fln_e':  'premium_eco_passengers',
                'wh_fleg_pax_fln_f' : 'first_passengers',
                'wh_fleg_pax_fln_c' : 'business_passengers',
                'wh_fleg_pax_fln_y' : 'eco_passengers',

                # Airport
                'flights_per_day':'flights_per_day',
                'previous_is_delayed':'previous_is_delayed',

                
                # LABELS
                'wh_fdel_delay_time':'delay_minutes',
                'is_delayed':'delayed',
                }

            

In [ ]:
merged_df = merged_df.rename(columns=map_columns)
merged_df = merged_df[map_columns.values()]

### Investigate relationship between these features

In [ ]:
merged_df.id_flight = merged_df.id_flight.astype(str)

In [ ]:
merged_df.describe(include='all')

### Let's study some correlations between numerical features

In [ ]:

#pd.set_option('display.max_columns', None)
#old_merged.corr(numeric_only=True)

In [ ]:
merged_df.corr(numeric_only=True)

It's interesting to see that there's nice correlation between CARGO DATA, NUMBER CHECKED BAGS (AND THEIR WEIGHTS), AND PASSENGER TYPE SEAT.
This makes sense because the longer the trip, intuitively it's more likely that people will bring a checked bag and will purchase first/business class.

Also, it interesting to see that (as expected) weight_checked_luggages and number_checked_luggages is correlated. We want to remove this since it might uniquely introduce MULTICOLLINEARITY.

It doesnt' seem, tho, that there is a direct correlation between these features and delay minutes. This makes sense because delay is the result of a combination of factors, and not uniquely dependet on this factor.

Maybe, introducing the weather information can give us more information about it. But let's check whether most of the delays are indeed caused by it. We can use the delay dataset and AITA codes to map the codes to delay.

It seems that it's not the weather is not the most  common cause of delay. Besides the code 0 which is specific to the airline, the other code 93 referes to aircraft rotation, late arrivial from another flight.
The other common cause is ATC capacity: Air Traffic Management System is its ability to provide Air navigation Services to a certain volume of air traffic, in line with the targeted high level of safety and without imposing significant operational, economic or environmental penalties under normal circumstances




In [ ]:
df_delay.groupby(['wh_fdel_delay_code','wh_fdel_delay_subcode']).count().loc[:, ['wh_fdel_leg_i']].sort_values(by = 'wh_fdel_leg_i', ascending=False).head(10)

# 0 (internal err) -> Not easy to understand what is is
# 93 (RA): Aircraft rotation, late arrival of aircraft from another flight or previous sector
# 81 (AT): ATC restriction en-route or capacity: aircraft on IFR flight plans, generally by centers, when these aircraft are operating between departure and destination terminal areas.
# 89 (AM): Restrictions at airport of departure,airport/runway closed due obstruction, industrial action, staff shortage, political unrest, noise abatemen
# 91 (RL): Passenger or Load Connection, awaiting load or passengers from another flight. Protection of stranded passengers onto a new flight.
# 83 (AE): ATC restriction at destination
# 85 (AS): Mandatory security
# 84 (AW): ATC restriction due to weather at destination
# 16 (PS): Commercial Publicity, Passenger Convenience, VIP, Press, Ground meals and missing personal items
# 2 Delay codes starting with 2 (cargo/mail)
# 41 (TD): Aircraft defects

### Let's study the relationship between categorical variables

In [ ]:
%matplotlib inline

In [ ]:
categorical_columns = merged_df.select_dtypes(include=['object']).iloc[:,1:].columns
categorical_columns

In [ ]:
sns.boxplot(x=categorical_columns[0], y='delay_minutes', data=merged_df) # WE REMOVE DC (THERE's NO delay data -> undersample)

In [ ]:
merged_df.season_trip = merged_df.season_trip.str.strip()

In [ ]:
sns.boxplot(y='delay_minutes', x='season_trip', data=merged_df, orient='v', showfliers = False) # doesn't seem to exists a direct link 


### Origin- Destination airports 

In [ ]:
most_common_airports_origin = list(merged_df.groupby('origin').count().sort_values(by='id_flight',ascending=False).index[:30])

In [ ]:
sns.set(rc={'figure.figsize':(20.7,10.27)})
sns.boxplot(y='delay_minutes', x =categorical_columns[1], data=merged_df.where(merged_df['origin'].isin(most_common_airports_origin)), orient='v', showfliers = False)

In [ ]:
most_common_airports_origin = list(merged_df.groupby('destination').count().sort_values(by='id_flight',ascending=False).index[:30])
sns.set(rc={'figure.figsize':(20.7,10.27)})
sns.boxplot(y='delay_minutes', x ='destination', data=merged_df.where(merged_df['destination'].isin(most_common_airports_origin)), orient='v', showfliers = False)

It looks like there's a relationship between the airports of origing and destination and the delay (it makes sense)

## Include information about time

In [ ]:
merged_df['day_of_week'] = merged_df.scheduled_time_departure.apply(lambda x: pd.to_datetime(x).day_of_week)
merged_df['day_of_year'] = merged_df.scheduled_time_departure.apply(lambda x: pd.to_datetime(x).day_of_year)
merged_df['month'] = merged_df.scheduled_time_departure.apply(lambda x: pd.to_datetime(x).month)
merged_df['year'] = merged_df.scheduled_time_departure.apply(lambda x: pd.to_datetime(x).year)

### Remove unuseful variables
- season_trip
- year (you don't want introduce bias)
- weight_checked_luggages ( multicollinearity)
- scheduled_time_departure
- carrier = DC

In [ ]:
merged_df = merged_df.drop(columns=['season_trip','id_flight'])

combine total_number_passenger

In [ ]:
merged_df['total_number_passengers'] = merged_df['first_passengers'] + merged_df['business_passengers'] + merged_df['eco_passengers'] + merged_df['premium_eco_passengers']


In [ ]:
merged_df = merged_df.drop(columns=['first_passengers','eco_passengers','business_passengers','premium_eco_passengers'])


In [ ]:
# remove cancelled flights to downsample majority class
merged_df = merged_df[~cancelled_flights_idx]

In [ ]:
# remvoe carrier DC to downsample mahority class
merged_df = merged_df[merged_df.carrier !='DC']

In [ ]:
# remove duplicate entries
merged_df.duplicated().sum() # 8604
merged_df = merged_df.drop_duplicates()

In [ ]:
merged_df.isna().sum() # check nan values

In [ ]:
#map label to 0 and 1
merged_df.delayed.unique()
merged_df['delayed'] = merged_df['delayed'].astype(int)

In [ ]:
cleaned_df = merged_df[[x for x in merged_df.columns if x !='delayed']+['delayed']] # add label at the end

In [ ]:
cleaned_df.describe(include='all')

In [ ]:
cleaned_df[cleaned_df.distance_trip == 0].groupby(by='delayed').count() # turn around flight (SHALL I REMOVE IT) ==> I add info in the db itself

cleaned_df['is_turnaround']  = (cleaned_df['distance_trip'] == 0).astype(int)

In [ ]:
#cleaned_df = cleaned_df[[x for x in cleaned_df.columns if x !='delayed']+['delayed']]
cleaned_df.previous_is_delayed = cleaned_df.previous_is_delayed.astype('category')

In [ ]:
cleaned_df.head()

In [ ]:
flights_inbetween = pd.read_csv('/Users/favea/Downloads/swiss-data/num_flights_within_hour.csv')
flights_inbetween.head()

In [ ]:
with wandb.init(project='swiss-delay-prediction', entity=None, job_type="processed-dataset") as run:
    table_merged = wandb.Table(dataframe=cleaned_df)


    # Create an artifact for our dataset
    dataset_artifact = wandb.Artifact(
        'dataset-cleaned', type='dataset',
        description='Table containing the cleaned dataset that can be use for training',
    )
    # Add the table to the artifact & log the artifact
    dataset_artifact.add(table_merged, 'data-table-delay-cleaned')


    # Add the 
    run.log_artifact(dataset_artifact)
    



In [ ]:
from sklearn.model_selection import train_test_split

## Split dataset into train,validation, and test set

In [ ]:
train_full, test = train_test_split(cleaned_df.values, test_size=.10,  shuffle = True, random_state=123456,stratify= cleaned_df.values[:,-1])
train, dev = train_test_split(train_full, test_size=.20,  shuffle = True, random_state=123456,stratify= train_full[:,-1])


In [ ]:
len(train), len(dev), len(test)

In [ ]:
train_df = pd.DataFrame(train, columns=cleaned_df.columns)
dev_df = pd.DataFrame(dev, columns=cleaned_df.columns)
test_df = pd.DataFrame(test, columns=cleaned_df.columns)

In [ ]:
out_path = '/Users/favea/Downloads/swiss-data'

In [ ]:
cleaned_df.to_csv(os.path.join(out_path,'cleaned_df.csv'))

In [ ]:
train_df.to_csv(os.path.join(out_path,'train_df.csv'))
dev_df.to_csv(os.path.join(out_path,'dev_df.csv'))
test_df.to_csv(os.path.join(out_path,'test_df.csv'))

In [ ]:
dev_df.head()

In [ ]:
with wandb.init(project='swiss-delay-prediction', entity=None, job_type="train-dev-test-split") as run:
    

    artifact = wandb.Artifact('stratified_split', type='dataset')
    artifact.add_file('/Users/favea/Downloads/swiss-data/train_df.csv')
    artifact.add_file('/Users/favea/Downloads/swiss-data/dev_df.csv')
    artifact.add_file('/Users/favea/Downloads/swiss-data/test_df.csv')

    run.log_artifact(artifact)
